# A mobile app for lottery addiction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

### Core Functions

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. The drawing is done without replacement, which means once a number is drawn, it's not put back in the set.

We'll use two principal function for calculate `factorial` and `combinations`.

In [3]:
def factorial(n):
    """
    Calculate the factorial of n (n!)
    Args:
        n: numeric value
    Returns:
        The n!.
    Examples
    ________
    >>> factorial(5)
    120
    """
    if n == 2:
        return 2
    return n * factorial(n - 1)

In [4]:
def combinations(n, k):
    """
    Calculate the combinations of n taking k objects.
    Args:
        n: number represent a group.
        k: number of takin objectcs from group.
    Returns:
        The nCk.
    Examples
    ________
    >>> combinations(52, 5)
    2598960.0
    """
    return factorial(n) / (factorial(k) * factorial(n - k))


In [17]:
def one_ticket_probability(numbers):
    outcomes = combinations(49, 6)
    successful_outcome = 1 / outcomes
    percentage_win = successful_outcome * 100
    print(f'The probability of the combination {numbers} winning the grand prize is {percentage_win} % (1 / i)')    

In [18]:
one_ticket_probability(['13', '17', '19', '10', '30', '43'])

7.151123842018516e-08
The probability of the combination ['13', '17', '19', '10', '30', '43'] winning the grand prize is 7.151123842018516e-06 % (1 / 0)


In [13]:
one_ticket_probability(['3', '1', '4', '1', '6', '5'])

The probability of the combination ['3', '1', '4', '1', '6', '5'] winning the grand prize is 7.151123842018516e-06 %.
